# 🌳 Day 4 · Forest Change Mapping
Students now connect tabular data to a global map. Maintain the loop cadence and emphasise metadata awareness.

## 🔄 How to Use This Solution
- Validate the geographic identifiers before plotting.
- Use the data card to discuss units (forest area as % of land).
- Encourage students to draft captions describing both the map and its limitations.

> ### 🗂️ Data Card — World Bank Forest Area
> - **Source:** World Bank (`AG.LND.FRST.ZS`) processed into long format.
> - **Temporal coverage:** 1990–2021.
> - **Metric:** Forest area as a percentage of total land area.
> - **Refresh cadence:** Annual; reshaped January 2024.
> - **Caveats:** Aggregated regions (e.g., “Africa Eastern and Southern”) mix countries; small islands may have noisy estimates.
> - **Ethics & framing:** Maps emphasise geography but hide absolute area; pair with annotations that prevent misinterpretation.

In [ ]:

from __future__ import annotations
from pathlib import Path
from typing import Mapping, Sequence

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

DATA_DIR = Path.cwd() / "data"
PLOTS_DIR = Path.cwd() / "plots"
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

sns.set_theme(style="whitegrid", context="notebook", palette="colorblind")
plt.rcParams.update({
    "figure.dpi": 120,
    "axes.titlesize": 18,
    "axes.labelsize": 13,
    "axes.titleweight": "semibold",
    "axes.grid": True,
})


def baseline_style() -> None:
    """Reset plot style to the shared course defaults."""
    sns.set_theme(style="whitegrid", context="notebook", palette="colorblind")
    plt.rcParams.update({
        "axes.grid": True,
        "axes.spines.top": False,
        "axes.spines.right": False,
        "figure.dpi": 120,
        "font.size": 12,
    })


def load_data(path: Path, *, read_kwargs: Mapping[str, object] | None = None) -> pd.DataFrame:
    read_kwargs = dict(read_kwargs or {})
    df = pd.read_csv(path, **read_kwargs)
    print(f"✅ Loaded {path.name} with shape {df.shape}")
    return df


def validate_columns(df: pd.DataFrame, required: Sequence[str]) -> None:
    missing = [col for col in required if col not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")
    print("✅ Column check passed:", ", ".join(required))


def expect_rows_between(df: pd.DataFrame, lower: int, upper: int) -> None:
    rows = len(df)
    if not (lower <= rows <= upper):
        raise ValueError(f"Expected between {lower} and {upper} rows, got {rows}")
    print(f"✅ Row count within expected range ({rows} rows)")


def quick_diagnose(df: pd.DataFrame, *, label: str = "Data preview", n: int = 5) -> None:
    print(f"
🔍 {label}")
    display(df.head(n))
    numeric = df.select_dtypes(include="number")
    if not numeric.empty:
        display(numeric.describe().T)
    nulls = df.isna().sum()
    if nulls.any():
        print("⚠️ Null values detected:
", nulls[nulls > 0])
    else:
        print("✅ No null values detected in this slice.")


def accessibility_check(ax: plt.Axes) -> None:
    title_ok = bool(ax.get_title())
    label_ok = bool(ax.get_xlabel()) and bool(ax.get_ylabel())
    if not (title_ok and label_ok):
        raise ValueError("Add a descriptive title and axis labels before proceeding.")
    xlabels = [tick.get_text() for tick in ax.get_xticklabels()]
    if len(xlabels) > 12:
        ax.tick_params(axis='x', labelrotation=35)
    print("✅ Accessibility check: title, labels, and readable ticks confirmed.")


def annotate_latest_point(ax: plt.Axes, x: float, y: float, text: str) -> None:
    ax.scatter([x], [y], color=ax.lines[0].get_color(), s=60, zorder=5)
    ax.annotate(
        text,
        xy=(x, y),
        xytext=(0.96, 0.85),
        textcoords="axes fraction",
        ha="right",
        arrowprops={"arrowstyle": "->", "color": "#333"},
        fontsize=11,
    )


def require_story_elements(story: Mapping[str, str]) -> None:
    required = ["claim", "evidence", "visual", "takeaway", "source", "units"]
    missing = [key for key in required if not story.get(key, "").strip()]
    if missing:
        raise ValueError(f"Fill in the storytelling scaffold: missing {missing}")
    print("✅ Story scaffold complete.")


def save_last_fig(filename: str, fig: plt.Figure | None = None) -> None:
    fig = fig or plt.gcf()
    if not fig.axes:
        raise ValueError("No Matplotlib figure found to save.")
    output_path = PLOTS_DIR / filename
    fig.savefig(output_path, bbox_inches="tight")
    print(f"💾 Figure saved to {output_path.relative_to(Path.cwd())}")


## Loop 1 · Load & Filter
Goal: load the long-form dataset and focus on a single recent year (2021) to produce a choropleth-ready table.

✅ **You should see:** Columns `Country Name`, `Country Code`, `Year`, `ForestPercent` with ~200 rows.

In [ ]:

baseline_style()

forest = load_data(DATA_DIR / "forest_area_long.csv")
validate_columns(forest, ["Country Name", "Country Code", "Year", "ForestPercent"])
expect_rows_between(forest, 5000, 8000)
quick_diagnose(forest.head(), label="Forest area (long format)")

forest_2021 = forest.query("Year == 2021").copy()
expect_rows_between(forest_2021, 180, 220)
quick_diagnose(forest_2021.head(), label="Forest area 2021")


## Loop 2 · Build Summary Table
Goal: prepare a teacher-friendly table that highlights leaders and laggards before plotting the map.

✅ **You should see:** Top/bottom five countries with clear units.

In [ ]:

leaders = forest_2021.nlargest(5, "ForestPercent")[['Country Name', 'ForestPercent']]
laggards = forest_2021.nsmallest(5, "ForestPercent")[['Country Name', 'ForestPercent']]
summary_table = pd.concat([
    leaders.assign(Category="Highest forest share"),
    laggards.assign(Category="Lowest forest share"),
])
summary_table["ForestPercent"] = summary_table["ForestPercent"].round(1)
display(summary_table)


## Loop 3 · Render the Choropleth
Goal: create an interpretable world map with title/subtitle scaffold plus accessibility checks (colourblind palette, readable legend).

In [ ]:

import plotly.express as px

TITLE = "Where Forests Cover the Land"
SUBTITLE = "Forest area as a share of land, 2021"
SOURCE = "Source: World Bank World Development Indicators (downloaded Jan 2024)"
UNITS = "Forest area (% of land)"

story = {
    "claim": "Large forest reserves cluster in South America and Central Africa, while arid nations remain sparse.",
    "evidence": f"Median forest cover globally is {forest_2021['ForestPercent'].median():.1f}%. Countries above 60% include {', '.join(leaders['Country Name'].head(3))}.",
    "visual": "Plotly choropleth map of 2021 forest share.",
    "takeaway": "Protecting high-cover regions matters, but nations with sparse forests require restoration strategies, not the same policy mix.",
    "source": SOURCE,
    "units": UNITS,
}
require_story_elements(story)

fig = px.choropleth(
    forest_2021,
    locations="Country Code",
    color="ForestPercent",
    hover_name="Country Name",
    color_continuous_scale="YlGn",
    range_color=(0, 80),
    title=f"{TITLE}<br><sup>{SUBTITLE}</sup>",
)
fig.update_layout(
    coloraxis_colorbar_title="% forest",
    margin=dict(l=0, r=0, t=80, b=0),
    annotations=[dict(text=SOURCE, x=0, y=0, xref="paper", yref="paper", showarrow=False, font=dict(size=11, color="#555"))],
)
fig.show()

story_df = pd.DataFrame([story]).T.rename(columns={0: "Story Scaffold"})
display(story_df)


## Loop 4 · Extension Prompts
- Compare 1990 vs 2021 to quantify change over time.
- Pair the map with a bar chart for selected regions to reinforce quantitative reading.
- Ask students to note what the map cannot tell us (e.g., biodiversity quality, deforestation rates).

In [ ]:

# Save a static image of the map using Plotly's write_image if kaleido is available.
try:
    import plotly.io as pio

    output_path = PLOTS_DIR / "day04_solution_plot.png"
    pio.write_image(fig, output_path, scale=2)
    print(f"💾 Figure saved to {output_path.relative_to(Path.cwd())}")
except Exception as exc:
    print("⚠️ Unable to export Plotly figure automatically:", exc)
